# Dimensões

In [0]:
CREATE OR REPLACE TABLE puc.gold.dim_tempo
USING DELTA
AS
SELECT DISTINCT
  to_date(data_solicitacao)                         AS dt_solicitacao,
  year(to_date(data_solicitacao))                   AS ano,
  month(to_date(data_solicitacao))                  AS mes,
  day(to_date(data_solicitacao))                    AS dia,
  quarter(to_date(data_solicitacao))                AS trimestre,
  date_format(to_date(data_solicitacao), 'yyyy-MM') AS ano_mes,
  dayofweek(to_date(data_solicitacao))              AS dia_semana
FROM puc.silver.base_pd
WHERE data_solicitacao IS NOT NULL;

In [0]:
CREATE OR REPLACE TABLE puc.gold.dim_localizacao
USING DELTA
AS
SELECT
  row_number() OVER (ORDER BY uf, regiao) AS sk_localizacao,
  uf,
  regiao
FROM (
  SELECT DISTINCT
    uf,
    regiao
  FROM puc.silver.base_pd
) x;

In [0]:
CREATE OR REPLACE TABLE puc.gold.dim_produto
USING DELTA
AS
SELECT
  row_number() OVER (ORDER BY produto) AS sk_produto,
  produto
FROM (
  SELECT DISTINCT produto
  FROM puc.silver.base_pd
) x;

In [0]:
CREATE OR REPLACE TABLE puc.gold.dim_canal
USING DELTA
AS
SELECT
  row_number() OVER (ORDER BY canal) AS sk_canal,
  canal
FROM (
  SELECT DISTINCT canal
  FROM puc.silver.base_pd
) x;

In [0]:
CREATE OR REPLACE TABLE puc.gold.dim_cliente
USING DELTA
AS
SELECT
  row_number() OVER (ORDER BY id_cliente) AS sk_cliente,
  id_cliente,
  idade,
  escolaridade,
  estado_civil,
  vinculo_emprego,
  setor,
  tempo_emprego_anos,
  tempo_conta_anos,
  usa_internet_banking,
  possui_cartao_credito,
  possui_investimentos,
  possui_seguro
FROM (
  SELECT
    id_cliente,
    max(idade)                  AS idade,
    max(escolaridade)           AS escolaridade,
    max(estado_civil)           AS estado_civil,
    max(vinculo_emprego)        AS vinculo_emprego,
    max(setor)                  AS setor,
    max(tempo_emprego_anos)     AS tempo_emprego_anos,
    max(tempo_conta_anos)       AS tempo_conta_anos,
    max(usa_internet_banking)   AS usa_internet_banking,
    max(possui_cartao_credito)  AS possui_cartao_credito,
    max(possui_investimentos)   AS possui_investimentos,
    max(possui_seguro)          AS possui_seguro
  FROM puc.silver.base_pd
  GROUP BY id_cliente
) x;

# Fatos

In [0]:
CREATE OR REPLACE TABLE puc.gold.fato_solicitacao_credito
USING DELTA
AS
SELECT
  -- Degenerate key do evento (boa prática p/ auditoria)
  sha2(concat_ws('||',
      cast(f.id_cliente as string),
      cast(to_date(f.data_solicitacao) as string),
      coalesce(f.produto,''),
      coalesce(f.canal,''),
      coalesce(f.uf,''),
      coalesce(cast(f.valor_emprestimo as string),'')
  ), 256) AS nk_evento,

  -- Chaves das dimensões
  dc.sk_cliente,
  dl.sk_localizacao,
  dp.sk_produto,
  dca.sk_canal,
  dt.dt_solicitacao,

  -- Medidas / atributos do evento (visão operacional)
  f.parcelas,
  f.valor_emprestimo,
  f.taxa_juros_mensal,
  f.valor_parcela,
  f.dti,
  f.score_credito,
  f.atrasos_passados,

  f.renda_mensal_atual,
  f.renda_mensal_anterior,
  f.qtd_produtos_bancarios,
  f.frequencia_transacoes,
  f.valor_emprestimos_anteriores,

  -- “label”/resultado (útil p/ análises)
  f.inadimplente

FROM puc.silver.base_pd f
JOIN puc.gold.dim_cliente      dc  ON f.id_cliente = dc.id_cliente
JOIN puc.gold.dim_localizacao  dl  ON f.uf = dl.uf AND f.regiao = dl.regiao
JOIN puc.gold.dim_produto      dp  ON f.produto = dp.produto
JOIN puc.gold.dim_canal        dca ON f.canal = dca.canal
JOIN puc.gold.dim_tempo        dt  ON to_date(f.data_solicitacao) = dt.dt_solicitacao

In [0]:
CREATE OR REPLACE TABLE puc.gold.fato_risco_cliente_mes
USING DELTA
AS
WITH base AS (
  SELECT
    dc.sk_cliente,
    dl.sk_localizacao,
    dp.sk_produto,
    dca.sk_canal,
    date_trunc('month', to_date(f.data_solicitacao)) AS dt_mes,

    -- risco/IFRS9-like
    f.pd_true,
    f.pd_model,
    f.ead,
    f.lgd,
    f.valor_recuperado,

    -- opcional: outcomes
    f.inadimplente
  FROM puc.silver.base_pd f
  JOIN puc.gold.dim_cliente      dc  ON f.id_cliente = dc.id_cliente
  JOIN puc.gold.dim_localizacao  dl  ON f.uf = dl.uf AND f.regiao = dl.regiao
  JOIN puc.gold.dim_produto      dp  ON f.produto = dp.produto
  JOIN puc.gold.dim_canal        dca ON f.canal = dca.canal
)
SELECT
  sha2(concat_ws('||',
      cast(sk_cliente as string),
      cast(dt_mes as string),
      cast(sk_produto as string),
      cast(sk_canal as string),
      cast(sk_localizacao as string)
  ), 256) AS nk_risco_mes,

  sk_cliente,
  sk_localizacao,
  sk_produto,
  sk_canal,
  dt_mes,

  -- Agregações de risco (mensal)
  avg(pd_model)              AS pd_model_media,
  avg(pd_true)               AS pd_true_media,
  sum(ead)                   AS ead_soma,
  avg(lgd)                   AS lgd_media,
  sum(valor_recuperado)      AS valor_recuperado_soma,

  -- Métrica clássica: Perda Esperada (EL = PD * EAD * LGD)
  sum(coalesce(pd_model,0) * coalesce(ead,0) * coalesce(lgd,0)) AS perda_esperada_soma,

  -- opcional: taxa de inadimplência no mês (para discussão)
  avg(cast(inadimplente as double)) AS inadimplencia_media

FROM base
GROUP BY
  sk_cliente, sk_localizacao, sk_produto, sk_canal, dt_mes;